# Working notebook read acqknowledge

In this notebook I am workingo on functions to read acknowledge files and generate stimlist from digital channels. Aim is to create analogical files to ones from brainvision.


In [62]:
import bioread
import mne
import numpy as np
from pandas import DataFrame

def code_generator(n,firstLetter = 'S'):
    return '{}{: 3d}'.format(firstLetter,n)

In [57]:
def create_stimlist_file_from_data(filename):
    data = bioread.read_file(filename)
    x = lambda a : a>0
    result = []
    print('creating stimlist...')
    for idx in range(2,len(data.channels[0].data)):
        uno = [str(int(x(n.data[idx]))) for n in list(reversed(data.channels[1:]))]
        duo = [str(int(x(n.data[idx-1]))) for n in list(reversed(data.channels[1:]))]
        uno = int(''.join(uno),2)
        duo = int(''.join(duo),2)
        if (uno>1 and duo<1):
            code = code_generator(uno,'S')
            result.append((code,idx))
    print('done!')
    return result

In [52]:
def convert_acq_mne(filename='analiza_follow up\JM22a_B140_TP6_evt.acq'):
    data = bioread.read_file(filename)
    n_channels = len(data.channels)
    sampling_freq = int(data.channels[0].samples_per_second)  # in Hertz
    channel_names = [n.name for n in data.channels]

    times = int(data.channels[0].samples_per_second)
    data = np.array([n.data for n in data.channels])
    info = mne.create_info(
        ch_names=channel_names, ch_types=['misc']*n_channels, sfreq=sampling_freq
    )
    print(info)    
    raw = mne.io.RawArray(data, info)
    return raw

In [68]:
raw = convert_acq_mne('analiza_follow up\JM22A_B004_TP6.acq')
stimlist = create_stimlist_file_from_data('analiza_follow up\JM22A_B004_TP6.acq')

DataFrame(stimlist).to_excel('JM22A_B004_TP6_eventlist.xlsx')

<Info | 7 non-empty values
 bads: []
 ch_names: EDA100C-MRI, Digital (STP Input 0), Digital (STP Input 1), ...
 chs: 9 misc
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 9
 projs: []
 sfreq: 2000.0 Hz
>
Creating RawArray with float64 data, n_channels=9, n_times=4072563
    Range : 0 ... 4072562 =      0.000 ...  2036.281 secs
Ready.
creating stimlist...
done!


In [69]:
raw

<RawArray | 9 x 4072563 (2036.3 s), ~279.7 MB, data loaded>